In [14]:
import scanpy as sc
import pandas as pd
import numpy as np
import sklearn.metrics
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
import copy

In [15]:
label = sc.read_csv('darmanis_label.csv',dtype="str")
adata = sc.read_csv("darmanis_counts.csv")
data = adata.to_df()

In [16]:
adata
# data

AnnData object with n_obs × n_vars = 466 × 22088

In [17]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

def build_dataset(adata_this,label_this,drop_number=1):



    adata=adata_this
    label=label_this.to_df()

    print("Dataset Size {}".format(len(adata)))

    assert len(adata)==len(label),"Data Length {} Does not Match Label Length {}".format(len(adata),len(label))

    set_label=np.unique(label.values)
    print(set_label)

    #Find the rare cell class
    count=[]
    for label_index in range(len(set_label)):
        # print(label.values==set_label[label_index])
        sum_number=sum(label.values==set_label[label_index])
        count.append(sum_number[0])
        print("Label {}: {}".format(set_label[label_index],sum_number))
    print(count)
    print(np.array(count))
    print(np.argmin(count))
    min_index=np.argsort(count)[:drop_number]
    print(min_index)
    min_label=[]
    for i in min_index:
        min_label.append(set_label[i])
    print(min_label)
    print("Rare Cell Name {}: {}".format(min_label,[count[count_i] for count_i in min_index]))

    # Build Class Mapping for cell class
    label_map={}
    count_map=0
    for label_index in range(len(set_label)):
        if not label_index in min_index:
            label_map[set_label[label_index]]=count_map
            count_map+=1
    print(label_map)
    count_map_temp=count_map
    for label_index in range(len(set_label)):
        if label_index in min_index:
            label_map[set_label[label_index]]=count_map_temp
            count_map_temp+=1
    # label_map[min_label]=count_map
    print(label_map)

    test_extract_ood = label.isin(min_label)
    label_ood = label.loc[test_extract_ood.values]
    X_test_ood=adata.loc[test_extract_ood.values]
    # print(label_ood.values[0])
    adata_id = adata.loc[~test_extract_ood.values]
    label_id = label.loc[~test_extract_ood.values]

    X_train, X_test, y_train, y_test = train_test_split(adata_id,label_id,random_state=42,stratify=label_id)
    print(X_train)
    print(y_train)
    return X_train,X_test,y_train,y_test,X_test_ood,label_ood
X_train,X_test,y_train,y_test,X_test_ood,label_ood=build_dataset(data,label,drop_number=1)
true_label=np.zeros(len(X_test)+len(X_test_ood))
true_label[:len(X_test)]+=1

Dataset Size 466
['"OPC"' '"astrocytes"' '"endothelial"' '"fetal_quiescent"'
 '"fetal_replicating"' '"hybrid"' '"microglia"' '"neurons"'
 '"oligodendrocytes"']
Label "OPC": [18]
Label "astrocytes": [62]
Label "endothelial": [20]
Label "fetal_quiescent": [110]
Label "fetal_replicating": [25]
Label "hybrid": [46]
Label "microglia": [16]
Label "neurons": [131]
Label "oligodendrocytes": [38]
[18, 62, 20, 110, 25, 46, 16, 131, 38]
[ 18  62  20 110  25  46  16 131  38]
6
[6]
['"microglia"']
Rare Cell Name ['"microglia"']: [16]
{'"OPC"': 0, '"astrocytes"': 1, '"endothelial"': 2, '"fetal_quiescent"': 3, '"fetal_replicating"': 4, '"hybrid"': 5, '"neurons"': 6, '"oligodendrocytes"': 7}
{'"OPC"': 0, '"astrocytes"': 1, '"endothelial"': 2, '"fetal_quiescent"': 3, '"fetal_replicating"': 4, '"hybrid"': 5, '"neurons"': 6, '"oligodendrocytes"': 7, '"microglia"': 8}
                               1/2-SBSRNA4  A1BG  A1BG-AS1  A1CF  A2LD1  \
GSM1658333_nochipID13.C75.csv          0.0   0.0       0.0   0.0

In [18]:
test_all=pd.concat([X_test,X_test_ood])
print(test_all)

                               1/2-SBSRNA4      A1BG  A1BG-AS1  A1CF  A2LD1  \
GSM1658101_nochipID3.C61.csv      0.000000  0.000000       0.0   0.0    0.0   
GSM1658350_nochipID4.C41.csv      0.000000  0.000000       0.0   0.0    0.0   
GSM1658047_nochipID2.C50.csv      0.027735  0.000000       0.0   0.0    0.0   
GSM1658327_nochipID13.C66.csv     0.000000  0.000000       0.0   0.0    0.0   
GSM1658003_nochipID15.C54.csv     0.000000  0.202021       0.0   0.0    0.0   
...                                    ...       ...       ...   ...    ...   
GSM1657925_1772078218.C88.csv     0.000000  0.000000       0.0   0.0    0.0   
GSM1657926_1772078218.C89.csv     0.000000  0.000000       0.0   0.0    0.0   
GSM1657927_1772078218.C93.csv     0.000000  0.000000       0.0   0.0    0.0   
GSM1657934_1772078236.C25.csv     0.000000  0.000000       0.0   0.0    0.0   
GSM1658116_nochipID5.C04.csv      0.000000  0.000000       0.0   0.0    0.0   

                                    A2M     A2ML1  

In [21]:
import scRare as sr

test_score = sr.scRare(test_all,X_train, copy.deepcopy(y_train),iteration_number=3000)



--------Start annotating----------
Computational unit be used is: cuda
Finish 11.0/3000
Finish 22.0/3000
Finish 33.0/3000
Finish 44.0/3000
Finish 55.0/3000
Finish 66.0/3000
Finish 77.0/3000
Finish 88.0/3000
Finish 99.0/3000
Finish 110.0/3000
Finish 121.0/3000
Finish 132.0/3000
Finish 143.0/3000
Finish 154.0/3000
Finish 165.0/3000
Finish 176.0/3000
Finish 187.0/3000
Finish 198.0/3000
Finish 209.0/3000
Finish 220.0/3000
Finish 231.0/3000
Finish 242.0/3000
Finish 253.0/3000
Finish 264.0/3000
Finish 275.0/3000
Finish 286.0/3000
Finish 297.0/3000
Finish 308.0/3000
Finish 319.0/3000
Finish 330.0/3000
Finish 341.0/3000
Finish 352.0/3000
Finish 363.0/3000
Finish 374.0/3000
Finish 385.0/3000
Finish 396.0/3000
Finish 407.0/3000
Finish 418.0/3000
Finish 429.0/3000
Finish 440.0/3000
Finish 451.0/3000
Finish 462.0/3000
Finish 473.0/3000
Finish 484.0/3000
Finish 495.0/3000
Finish 506.0/3000
Finish 517.0/3000
Finish 528.0/3000
Finish 539.0/3000
Finish 550.0/3000
Finish 561.0/3000
Finish 572.0/3000
Fi

In [22]:
auroc=sklearn.metrics.roc_auc_score(true_label,test_score)
print(auroc)

0.9032079646017699
